In [ ]:
!pip install -U --pre efficientnet >> /dev/null
!pip install tensorflow-addons

In [ ]:
! pip install gradio

In [ ]:

eb5model_filepath ='/kaggle/input/haminterfacetestdata/E5/HAMe5fold-2.h5'
eb5eb5model_filepath ='/kaggle/input/haminterfacetestdata/e5e5distV2(to temp)/Disfold-2.h5'
eb5eb5bestmodel_filepath ='/kaggle/input/haminterfacetestdata/E5E5_aplha-0-425-Notemp/Dis_v22E5E5NoTemp_0_425_Alpha_fold-2.h5'

In [ ]:
import os
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D
import matplotlib.pyplot as plt
from keras import backend as K
import numpy as np
#from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from PIL import Image, ImageChops
import random
#import gradio as gr
import pandas as pd
import cv2
import tensorflow as tf
from kaggle_datasets import KaggleDatasets
from functools import partial
import sklearn
from tqdm import tqdm_notebook as tqdm
import gc
from keras.models import load_model
#from focal_loss import BinaryFocalLoss
%matplotlib inline

In [ ]:
#classes = ['Melanocytic nevus', 'Melanoma', 'Benign keratosis', 'Dermatofibroma', 'Squamous cell carcinoma', 'Basal cell carcinoma', 'Vascular lesion', 'Actinic keratosis']
classes = ['Melanoma','Melanocytic nevus','Basal cell carcinoma','Actinic keratosis', 'Benign keratosis','Dermatofibroma','Vascular lesion']
#CLASSES = ['mel', 'nv', 'bcc', 'akiec', 'bkl', 'df', 'vasc']

In [ ]:

from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPooling2D, GlobalAveragePooling2D, Add, Dense
from tensorflow.keras.models import Model
import tensorflow_addons as tfa


In [ ]:
import efficientnet.tfkeras as efn
EFNS = [efn.EfficientNetB0, efn.EfficientNetB1, efn.EfficientNetB2, efn.EfficientNetB3, efn.EfficientNetB4, efn.EfficientNetB5, efn.EfficientNetB6, efn.EfficientNetB7]
def get_EFF_NET2(dim=384,output_bias=None):
    if output_bias is not None:
        output_bias = tf.keras.initializers.Constant(output_bias)
    
    inp = tf.keras.layers.Input(shape=(dim,dim,3), name='inp')
    base = EFNS[2](input_shape=(dim,dim,3),weights='noisy-student',include_top=False)
    base.trainable = True
    x = base(inp)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(7,activation='softmax', bias_initializer=output_bias)(x)
    model = tf.keras.Model(inputs=[inp],outputs=[x])
    return model
def get_EFF_NET5(dim=384,output_bias=None):
    if output_bias is not None:
        output_bias = tf.keras.initializers.Constant(output_bias)
    
    inp = tf.keras.layers.Input(shape=(dim,dim,3), name='inp')
    base = EFNS[5](input_shape=(dim,dim,3),weights='noisy-student',include_top=False)
    base.trainable = True
    x = base(inp)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(7,activation='softmax', bias_initializer=output_bias)(x)
    model = tf.keras.Model(inputs=[inp],outputs=[x])
    opti = tfa.optimizers.RectifiedAdam(learning_rate=0.00032, total_steps=10000,
                               warmup_proportion=0.1, min_lr=1e-7)
    loss= tf.keras.losses.SparseCategoricalCrossentropy()
    METRICS = [tf.keras.metrics.SparseCategoricalAccuracy()]
    model.compile(optimizer=opti, loss=loss, metrics=METRICS)
    return model


def get_EFF_NET7(dim=384,output_bias=None):
    if output_bias is not None:
        output_bias = tf.keras.initializers.Constant(output_bias)
    
    inp = tf.keras.layers.Input(shape=(dim,dim,3), name='inp')
    base = EFNS[7](input_shape=(dim,dim,3),weights='noisy-student',include_top=False)
    base.trainable = True
    x = base(inp)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(7,activation='softmax', bias_initializer=output_bias)(x)
    model = tf.keras.Model(inputs=[inp],outputs=[x])
    return model

In [ ]:
# modeleb2 = get_EFF_NET2()
# modeleb2.load_weights(eb2model_filepath)
# #modeleb2=tf.keras.models.load_model(eb2model_filepath)
# modeleb5 = get_EFF_NET5()
# modeleb5.load_weights(eb5model_filepath)
# modeleb7 = get_EFF_NET7()
# modeleb7.load_weights(eb7model_filepath)

modeleb5 = get_EFF_NET5()
modeleb5.load_weights(eb5model_filepath)


In [ ]:
class Distiller(tf.keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.5,
        temperature=1,
    ):
        """ Configure the distiller.
        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to distillation_loss_fn and 1-alpha to student_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature
    def train_step(self, data):
        # Unpack data
        x, y_ = data
        y = y_
        #y = tf.reshape(y_, (-1,7))
        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)


            loss_object = tf.keras.losses.SparseCategoricalCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
            per_example_loss = loss_object(y, student_predictions)
            student_loss = tf.nn.compute_average_loss(per_example_loss, global_batch_size=BATCH_SIZES[fold]*REPLICAS)
            
            teacher_logit = (tf.math.log(teacher_predictions) - tf.math.log(1-teacher_predictions))
            teacher_predictions_soft = tf.math.sigmoid(teacher_logit/self.temperature)
            student_logit = (tf.math.log(student_predictions) - tf.math.log(1-student_predictions))
            student_predictions_soft = tf.math.sigmoid(student_logit/self.temperature)


            # L_soft
            per_example_loss_ = self.distillation_loss_fn(
                teacher_predictions,
                student_predictions,
            )
            distillation_loss = tf.nn.compute_average_loss(per_example_loss_, global_batch_size=BATCH_SIZES[fold]*REPLICAS)

#           Lstudent = αKD · Lsoft + (1 − αKD) · Lhard
            loss = (self.alpha) * distillation_loss + (1 - self.alpha) * student_loss



        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)


        self.optimizer.apply_gradients(zip(gradients, trainable_vars))


        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results
    
    def test_step(self, data):
        # Unpack the data
        x, y_ = data
        y = y_
        #y = tf.reshape(y_, (-1,7))
        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        #student_loss = self.student_loss_fn(y, y_prediction)
        
        loss_object = tf.keras.losses.SparseCategoricalCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
        per_example_loss = loss_object(y, y_prediction)
        student_loss = tf.nn.compute_average_loss(per_example_loss, global_batch_size=BATCH_SIZES[fold]*REPLICAS)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results

    def call(self, data):
        y_pred = self.student(data) 
        return y_pred

    @tf.function
    def distributed_train_step(dist_inputs):
        per_replica_losses = strategy.run(train_step, args=(dist_inputs,))
        return strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_losses,
                             axis=None)

    @tf.function
    def distributed_test_step(dist_inputs):
        return strategy.run(test_step, args=(dist_inputs,))
    
opt = tfa.optimizers.RectifiedAdam(lr=0.00032, total_steps=10000,
                               warmup_proportion=0.1, min_lr=1e-7)
opti = tfa.optimizers.Lookahead(opt, sync_period=5, slow_step_size=0.8)
loss= tf.keras.losses.SparseCategoricalCrossentropy()
METRICS = [tf.keras.metrics.SparseCategoricalAccuracy()]
teacher = get_EFF_NET5()#dim=IMG_SIZES[fold],ef=5, output_bias = None)
student = get_EFF_NET5()#dim=IMG_SIZES[fold],ef=0, output_bias = None)
distiller = Distiller(student=student, teacher=teacher)
student_scratch = tf.keras.models.clone_model(student)
# distillation_loss_fn22 = tf.keras.losses.KLDivergence(reduction=tf.keras.losses.Reduction.NONE)

distiller.compile(
            optimizer=opti,
            metrics=METRICS,
            student_loss_fn=loss,
            distillation_loss_fn=tf.keras.losses.KLDivergence(
                        reduction=tf.keras.losses.Reduction.NONE),
            alpha=0.40,
            temperature=1,)
distiller.built = True

In [ ]:
modeleb5eb5 = Distiller(student=student, teacher=teacher)
student_scratch = tf.keras.models.clone_model(student)
# distillation_loss_fn22 = tf.keras.losses.KLDivergence(reduction=tf.keras.losses.Reduction.NONE)

modeleb5eb5.compile(
            optimizer=opti,
            metrics=METRICS,
            student_loss_fn=loss,
            distillation_loss_fn=tf.keras.losses.KLDivergence(
                        reduction=tf.keras.losses.Reduction.NONE),
            alpha=0.40,
            temperature=1,)
modeleb5eb5.built = True
modeleb5eb5.load_weights(eb5eb5model_filepath)

In [ ]:
modeleb5eb5best = Distiller(student=student, teacher=teacher)
student_scratch = tf.keras.models.clone_model(student)
# distillation_loss_fn22 = tf.keras.losses.KLDivergence(reduction=tf.keras.losses.Reduction.NONE)

modeleb5eb5best.compile(
            optimizer=opti,
            metrics=METRICS,
            student_loss_fn=loss,
            distillation_loss_fn=tf.keras.losses.KLDivergence(
                        reduction=tf.keras.losses.Reduction.NONE),
            alpha=0.40,
            temperature=1,)
modeleb5eb5best.built = True
modeleb5eb5best.load_weights(eb5eb5bestmodel_filepath)

In [ ]:
import gradio as gr

In [ ]:
def image_classifier(inp,val):
    remarks=""
    output = ""
    #print(type(inp))
    #print(val)
    #print("H1")
    if inp is None:
        remarks="Please Select a valid input image"
    else:
        img = cv2.cvtColor(np.array(inp), cv2.COLOR_RGB2BGR)
        img = cv2.resize(img, (384, 384))
        img = img/255
        img = np.expand_dims(img, axis = 0)
        
        if val == "EfficientnetB5Only":
            print("here")
            pre=modeleb5.predict(img)[0]
            print(pre)
            max_valuei = np.argmax(pre)
            output = classes[max_valuei]
            remarks = ' , '.join([str(elem) for elem in pre ])
            remarks = "Model Output : " +remarks
        elif val == "EfficientnetB5StudentB5Teacher":
            pre=modeleb5eb5.predict(img)[0]
            max_valuei = np.argmax(pre)
            output = classes[max_valuei]
            remarks = ' , '.join([str(elem) for elem in pre ])
            remarks = "Model Output : " +remarks
        else:
            pre=modeleb5eb5best.predict(img)[0]
            max_valuei = np.argmax(pre)
            output = classes[max_valuei]
            remarks = ' , '.join([str(elem) for elem in pre ])
            remarks = "Model Output : " +remarks
#         else:
#             pre=modelcus.predict(img)[0]
#             max_valuei = np.argmax(pre)
#             output = classes[max_valuei]
#             remarks = ' , '.join([str(elem) for elem in pre ])
#             remarks = "Model Output : " +remarks
            #stats,remarks= use_model1(inp)
#         else:
#             print("HII")
            #stats,remarks= use_model2(inp)
            #remarks=" Pred "
            #stats={'Melanoma':0.0,'Non-Melanoma':0.0}
        #rem#arks=str(type(inp))
        #print(val)
    #     #call model
    #     pass
    #     stats['melanoma']=0.9
    #     stats['Non-melanoma']=0.1
        
    #print("going to return")
    #print(stats)
    #print(remarks)   
    return remarks,output

In [ ]:
demo = gr.Interface(fn=image_classifier, 
                    inputs=[gr.Image(type="pil"),gr.Radio(["EfficientnetB5Only", "EfficientnetB5StudentB5Teacher", "EfficientnetB5StudentB5TeacherRefined"],value="EfficientnetB5StudentB5TeacherRefined"),], 
                    outputs=[gr.Textbox(placeholder="Remarks",label="Output Remarks"),gr.Label(value="Output",label="Classification Output")],
                    title="MultiClass Skin Lesion Classification",
                    description="Please upload a valid skin lesion image,select the required model and press submit",
                    article="By Kiran Thomas Cherian CED18I028",
                    allow_flagging="never",
                    css="footer {visibility: hidden}"
                    )
                    
demo.launch(share=True,debug=True)

In [ ]:
# def our_cnn(dim=384, num_classes=8):
#     input_shape = (dim, dim, 3)
#     num_classes = 8

#     model = tf.keras.Sequential([
#         layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
#         layers.BatchNormalization(),
#         layers.Conv2D(32, (3, 3), activation='relu'),
#         layers.BatchNormalization(),
#         layers.MaxPooling2D((2, 2)),
#         layers.Dropout(0.25),

#         layers.Conv2D(64, (3, 3), activation='relu'),
#         layers.BatchNormalization(),
#         layers.Conv2D(64, (3, 3), activation='relu'),
#         layers.BatchNormalization(),
#         layers.MaxPooling2D((2, 2)),
#         layers.Dropout(0.25),

#         layers.Conv2D(128, (3, 3), activation='relu'),
#         layers.BatchNormalization(),
#         layers.Conv2D(128, (3, 3), activation='relu'),
#         layers.BatchNormalization(),
#         layers.MaxPooling2D((2, 2)),
#         layers.Dropout(0.25),

#         layers.Conv2D(256, (3, 3), activation='relu'),
#         layers.BatchNormalization(),
#         layers.Conv2D(256, (3, 3), activation='relu'),
#         layers.BatchNormalization(),
#         layers.MaxPooling2D((2, 2)),
#         layers.Dropout(0.25),

#         layers.Flatten(),
#         layers.Dense(512, activation='relu'),
#         layers.BatchNormalization(),
#         layers.Dropout(0.5),
#         layers.Dense(num_classes, activation='softmax')
#     ])
    return model